# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [12]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [13]:
df = spark_session.read.option("inferSchema", "true").csv("/data/movielens/ratings_100k.csv", header=True)


In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [14]:
from pyspark.mllib.recommendation import Rating

fold_count = 3

In [15]:
ratings = df.rdd.map(lambda row : Rating(row[0],row[1],row[2]))

r_array = ratings.randomSplit([0.33,0.33,0.34])


Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [16]:
rank = 20
iterations = 5
lambda_ = 0.02

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [22]:
scores = 0

In [23]:
from pyspark.mllib.recommendation import ALS

In [24]:
fold_count = 3

for test_fold_index in range(fold_count):
    
    train_index_1 = (test_fold_index+1)%fold_count
    train_index_2 = (test_fold_index+2)%fold_count    
    
    test_data = r_array[test_fold_index]
    train_data = r_array[train_index_1].union(r_array[train_index_2])
    
    model  = ALS.train(train_data, rank, iterations, lambda_)
    
    predictions = model.predictAll(test_data.map(lambda r : (r[0],r[1])))
    
    ratesAndPreds = predictions.map(lambda r : ((r[0],r[1]), r[2])).join(test_data.map(lambda r : ((r[0],r[1]), r[2]))).values()
                                          
    rmse = (ratesAndPreds.map(lambda r: (r[0] - r[1])**2).mean())**0.5
                                    
    scores += rmse
                                                                              
    
score = scores/fold_count 

Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [26]:
score

1.2985392814732692